# Triangle Meshes

Along with [points](2_Points.ipynb), [timeseries](3_Timeseries.ipynb), [trajectories](4_Trajectories.ipynb), and [rasters](5_Rasters.ipynb), Datashader can rasterize large triangular meshes, such as those often used to simulate data on an irregular grid:

<img src="../assets/images/chesbay_detail.png" width="500" height="500" style="border-width: 1px; border-style: solid;">

Any polygon can be represented as a set of triangles, and any shape can be approximated by a polygon, so the triangular-mesh support has many potential uses. 

In each case, the triangular mesh represents (part of) a *surface*, not a volume, and so the result fits directly into a 2D plane rather than requiring 3D rendering. This process of rasterizing a triangular mesh means generating values along specified regularly spaced intervals in the plane.  These examples from the [Direct3D docs](https://msdn.microsoft.com/en-us/library/windows/desktop/cc627092.aspx) show how this process works, for a variety of edge cases:
<img width=500 src="https://msdn.microsoft.com/dynimg/IC520311.png"/>

This diagram uses "pixels" and colors (grayscale), but for datashader the raster is more precisely interpreted as a 2D array with bins, not pixels, because the values involved are numeric rather than colors.  (With datashader, colors are assigned only in the later "shading" stage, not during rasterization itself.) As shown in the diagram, a pixel (bin) is treated as belonging to a given triangle if its center falls either inside that triangle or along its top or left edge.

The specific algorithm used to do so is based on the approach of [Pineda (1998)](http://people.csail.mit.edu/ericchan/bib/pdf/p17-pineda.pdf), which has the following features:
  * Classification of pixels relies on triangle convexity
  * Embarrassingly parallel linear calculations
  * Inner loop can be calculated incrementally, i.e. with very "cheap" computations
  
and a few assumptions: 
  * Triangles should be non overlapping (to ensure repeatable results for different numbers of cores)
  * Triangles should be specified consistently either in clockwise or in counterclockwise order of vertices (winding). 
  
Like the rest of Datashader, trimesh rasterization does not (yet) use the GPU; it's fast because of [Numba](http://numba.pydata.org) compiling Python into SIMD machine code instructions.

## Tiny example

To start with, let's generate a tiny set of 10 vertices at random locations:

In [ ]:
import numpy as np, datashader as ds, pandas as pd
import datashader.utils as du, datashader.transfer_functions as tf
from scipy.spatial import Delaunay

n = 10
np.random.seed(2)

x = np.random.uniform(size=n)
y = np.random.uniform(size=n)
z = np.random.uniform(0,1.0,x.shape)

pts = np.stack((x,y,z)).T
verts = pd.DataFrame(np.stack((x,y,z)).T, columns=['x', 'y' , 'z'])

Here we have a set of random x,y locations and associated z values.  We can see the numeric values with "head" and plot them (with color for z) using datashader's usual points plotting:

In [ ]:
cvs = ds.Canvas(plot_height=400,plot_width=400)

tf.Images(verts.head(15), tf.spread(tf.shade(cvs.points(verts, 'x', 'y', agg=ds.mean('z')), name='Points')))

To make a trimesh, we need to connect these points together into a non-overlapping set of triangles.  One well-established way of doing so is [Delaunay triangulation](https://en.wikipedia.org/wiki/Delaunay_triangulation):

In [ ]:
def triangulate(vertices, x="x", y="y"):
    """
    Generate a triangular mesh for the given x,y,z vertices, using Delaunay triangulation.
    For large n, typically results in about double the number of triangles as vertices.
    """
    triang = Delaunay(vertices[[x,y]].values)
    print('Given', len(vertices), "vertices, created", len(triang.simplices), 'triangles.')
    return pd.DataFrame(triang.simplices, columns=['v0', 'v1', 'v2'])

In [ ]:
%time tris = triangulate(verts)

The result of triangulation is a set of triangles, each composed of three indexes into the vertices array.  The triangle data can then be visualized by datashader's ``trimesh()`` method:

In [ ]:
tf.Images(tris.head(15), tf.shade(cvs.trimesh(verts, tris)))

By default, datashader will rasterize your trimesh using z values [linearly interpolated between the z values that are specified at the vertices](https://en.wikipedia.org/wiki/Barycentric_coordinate_system#Interpolation_on_a_triangular_unstructured_grid).  The shading will then show these z values as colors, as above.  You can enable or disable interpolation as you wish:

In [ ]:
from colorcet import rainbow as c
tf.Images(tf.shade(cvs.trimesh(verts, tris, interpolate='nearest'), cmap=c, name='10 Vertices'),
          tf.shade(cvs.trimesh(verts, tris, interpolate='linear'),  cmap=c, name='10 Vertices Interpolated'))

## More complex example

The small example above should demonstrate how triangle-mesh rasterization works, but in practice datashader is intended for much larger datasets. Let's consider a sine-based function `f` whose frequency varies with radius:

In [ ]:
rad = 0.05,1.0

def f(x,y):
    rsq = x**2+y**2
    return np.where(np.logical_or(rsq<rad[0],rsq>rad[1]), np.nan, np.sin(10/rsq))

We can easily visualize this function by sampling it on a raster with a regular grid:

In [ ]:
n = 400

ls  = np.linspace(-1.0, 1.0, n)
x,y = np.meshgrid(ls, ls)
img = f(x,y)

raster = tf.shade(tf.Image(img, name="Raster"))
raster

However, you can see pronounced aliasing towards the center of this function, as the frequency starts to exceed the sampling density of the raster.  Instead of sampling at regularly spaced locations like this, let's try evaluating the function at random locations whose density varies towards the center:

In [ ]:
def polar_dropoff(n, r_start=0.0, r_end=1.0):
    ls = np.linspace(0, 1.0, n)
    ex = np.exp(2-5*ls)/np.exp(2)
    radius = r_start+(r_end-r_start)*ex
    theta  = np.random.uniform(0.0,1.0, n)*np.pi*2.0
    x = radius * np.cos( theta )
    y = radius * np.sin( theta )
    return x,y

x,y = polar_dropoff(n*n, np.sqrt(rad[0]), np.sqrt(rad[1]))
z = f(x,y)

verts = pd.DataFrame(np.stack((x,y,z)).T, columns=['x', 'y' , 'z'])

We can now plot the x,y points and optionally color them with the z value (the value of the function f(x,y)):

In [ ]:
cvs = ds.Canvas(plot_height=400,plot_width=400)

tf.Images(tf.shade(cvs.points(verts, 'x', 'y'), name='Points'),
          tf.shade(cvs.points(verts, 'x', 'y', agg=ds.mean('z')), name='PointsZ'))

The points are clearly covering the area of the function that needs dense sampling, and the shape of the function can (roughly) be made out when the points are colored in the plot.  But let's go ahead and triangulate so that we can interpolate between the sampled values for display:

In [ ]:
%time tris = triangulate(verts)

And let's pre-compute the combined mesh data structure for these vertices and triangles, which for very large meshes (much larger than this one!) would save plotting time later:

In [ ]:
%time mesh = du.mesh(verts,tris)

This mesh can be used for all future plots as long as we don't change the number or ordering of vertices or triangles, which saves time for much larger grids.

We can now plot the trimesh to get an approximation of the function with noisy sampling locally to disrupt the interference patterns observed in the regular-grid version above and preserve fidelity where it is needed. (Usually one wouldn't do this just for the purposes of plotting a function, since the eventual display on a screen is a raster image no matter what, but having a variable grid is crucial if running a simulation where fine detail is needed only in certain regions.)

In [ ]:
tf.shade(cvs.trimesh(verts, tris, mesh=mesh))

The fine detail in the heavily sampled regions is visible when zooming in closer (without resampling the function):

In [ ]:
tf.Images(*([tf.shade(ds.Canvas(x_range=r, y_range=r).trimesh(verts, tris, mesh=mesh))
            for r in [(0.1,0.8), (0.14,0.4), (0.15,0.2)]]))

Notice that the central disk is being filled in above, even though the function is not defined in the center.  That's a limitation of Delaunay triangulation, which will create convex regions covering the provided vertices.  You can use other tools for creating triangulations that have holes, align along certain regions, have specified densities, etc., such as [MeshPy](https://mathema.tician.de/software/meshpy) (Python bindings for [Triangle](http://www.cs.cmu.edu/~quake/triangle.html)).


### Aggregation functions

Like other datashader methods, the ``trimesh()`` method accepts an ``agg`` argument (defaulting to ``mean()``) for a reduction function that determines how the values from multiple triangles will contribute to the value of a given pixel:

In [ ]:
tf.Images(tf.shade(cvs.trimesh(verts, tris, mesh=mesh, agg=ds.mean('z')),name='mean'),
          tf.shade(cvs.trimesh(verts, tris, mesh=mesh, agg=ds.max('z')), name='max'),
          tf.shade(cvs.trimesh(verts, tris, mesh=mesh, agg=ds.min('z')), name='min'))

The three plots above should be nearly identical, except near the center disk where individual pixels start to have contributions from a large number of triangles covering different portions of the function space.  In this inner ring, ``mean`` reports the average value of the surface inside that pixel, ``max`` reports the maximum value of the surface (hence being darker values in this color scheme), and ``Min`` reports the minimum value contained in each pixel.  The ``min`` and ``max`` reductions are useful when looking at a very large mesh, revealing details not currently visible. For instance, if a mesh has a deep but very narrow trough, it will still show up in the ``min`` plot regardless of your raster's resolution, while it might be missed on the ``mean`` plot.  

Other reduction functions are useful for making a mask of the meshed area (``any``), for showing how many triangles are present in a given pixel (``count``), and for reporting the diversity of values within each pixel (``std`` and ``var``):

In [ ]:
tf.Images(tf.shade(cvs.trimesh(verts, tris, mesh=mesh, agg=ds.any('z')), name='any'),
          tf.shade(cvs.trimesh(verts, tris, mesh=mesh, agg=ds.count()),  name='count'),
          tf.shade(cvs.trimesh(verts, tris, mesh=mesh, agg=ds.std('z')), name='std')).cols(3)

# Interactive plots

By their nature, fully exploring irregular grids needs to be interactive, because the resolution of the screen and the visual system are fixed.  Trimesh renderings can be generated as above and then displayed by Bokeh, or they can be handled entirely within HoloViews.  First, Bokeh:

### Bokeh

In [ ]:
from datashader.bokeh_ext import InteractiveImage
import bokeh.plotting as bp
bp.output_notebook()

def render_image(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.trimesh(verts, tris, mesh=mesh)
    return ds.transfer_functions.shade(agg)

p = bp.figure(tools='pan,wheel_zoom,reset', x_range=(-1, 1), y_range=(-1, 1), plot_width=600, plot_height=600)
InteractiveImage(p, callback=render_image)

Here, if you enable the wheel zoom tool and you have a live Python process (not just a static web page), you can zoom in to any location in this mesh and see the full data available. If you are just looking at the static web page, eventually you will see individual pixels in the original datashaded rasterized plot, not the full trimesh available.

### HoloViews

[HoloViews](http://holoviews.org) is designed to make working with data easier, including large or small trimeshes. With HoloViews, you first declare a ``hv.Trimesh`` object, then you apply the ``datashade()`` (or just ``aggregate()``) operation if the data is large enough to require datashader.  Notice that HoloViews expects the triangles and vertices in the *opposite* order as datashader's ``cvs.trimesh()``, because the vertices are optional for HoloViews:

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade
hv.extension("bokeh")

wireframe = datashade(hv.TriMesh((tris,verts), label="Wireframe").edgepaths)
trimesh   = datashade(hv.TriMesh((tris,hv.Points(verts, vdims='z')), label="TriMesh"), aggregator=ds.mean('z'))

(wireframe + trimesh).opts(
    opts.RGB(width=400, height=400))

Here you can zoom in on either of these plots, but they will only update if you have a live Python server (not a static web page.  The Wireframe plot will initially look like a collection of dots (as the triangles are all tiny), but zooming in will reveal the shape.  Notice how a few of the "wires" cross the center, because Delaunay triangulation has filled in the central region; other techniques as mentioned previously would be needed to avoid those.

For examples of Datashader's trimesh in use, see the [Chesapeake and Delaware Bays](../topics/bay_trimesh.ipynb) notebook:

<img src="../assets/images/chesapeake_farout.png" width="600">